In [1]:
from langchain import LLMMathChain, SelfAskWithSearchChain, OpenAI, Prompt, LLMChain, SerpAPIChain

In [11]:
template = """Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events. The input to this should be in the form of question.
Calculator: useful for when you need to answer questions about math

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of Search/Calculator
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {question}"""

In [12]:
prompt = Prompt(input_variables=["question"], template=template)

In [13]:
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain(llm=llm)

In [5]:
llm = OpenAI(temperature=0)
search = SerpAPIChain()

self_ask_with_search = SelfAskWithSearchChain(llm=llm, search_chain=search)

In [11]:
question = "What is thirty seven raised to the 0.23 power?"
question = "Who is Kim Kardashian dating?"

In [7]:
37**0.23

2.294491200736535

In [12]:
llm(question)

'\n\nKim Kardashian is currently dating Kanye West.'

In [33]:
llm_math_chain.run(question)

'Answer: 2.294491200736535\n'

In [49]:
p1 = prompt.format(question=question)

In [50]:
o1 = llm(p1, stop=["\nObservation"])

In [51]:
ps = [p for p in o1.split('\n') if p]

In [52]:
assert ps[-1].startswith('Action Input: ')
assert ps[-2].startswith('Action: ')

In [58]:
action = ps[-2][len('Action: '):]
action_input = ps[-1][len('Action Input: '):]

In [59]:
chain = {"Calculator": llm_math_chain}[action]

In [61]:
ca = chain.run(action_input)

In [62]:
question1 = question + o1 + f"\nObservation: {ca}"

In [65]:
p2 = prompt.format(question=question1)
llm(p2, stop=["\nObservation"])

'\nThought: I now know the final answer.\nFinal Answer: 2.294491200736535'

In [27]:
question = "What is the age of the person Kim Kardashian is dating raised to the 0.23 power?"
#question = "What is thirty seven raised to the 0.23 power?"

In [28]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
document = None
_input = f"{question}\nThought:"
while True:
    observation = llm_chain.predict(question=_input, stop=["\nObservation"])
    print(observation)
    ps = [p for p in observation.split('\n') if p]
    if ps[-1].startswith("Final Answer"):
        break
    assert ps[-1].startswith('Action Input: ')
    assert ps[-2].startswith('Action: ')
    action = ps[-2][len('Action: '):]
    action_input = ps[-1][len('Action Input: '):]
    chain = {"Calculator": llm_math_chain.run, 'Search': SerpAPIChain().search}[action]
    ca = chain(action_input)
    print(ca)
    _input = _input + observation + f"\nObservation: {ca}\nThought:"

 I need to find the age of the person Kim Kardashian is dating
Action: Search
Action Input: "Kim Kardashian boyfriend"
Kim Kardashian's boyfriend Pete Davidson has revealed his hopes to one day become a dad, admitting he's 'so excited' to have children.
 I need to find the age of Pete Davidson
Action: Search
Action Input: "Pete Davidson age"
28 years
 I need to calculate 28 to the 0.23 power
Action: Calculator
Action Input: 28^0.23
Answer: 2.1520202182226886

 I now know the final answer
Final Answer: 2.1520202182226886


In [8]:
_input

'Who is Kim Kardashian dating?\n\nThought: I should look up who Kim Kardashian is dating.\nAction: Search\nAction Input: Kim Kardashian dating\nObservation:  Kim Kardashian and Pete Davidson'

In [9]:
observation

' are dating.'

In [29]:
28**.23

2.1520202182226886